## <p style="text-align:center;color:midnightblue; font-size:190%;font-family:times new roman;"><u>Management Program Curriculum Design<u\></p>

## <p style="text-align:center;color:black; font-size:150%;font-family:times new roman;">Master of Data Science and Analytics</p>

### This is same as we did for 'Technical Data Science Program Curriculum Design' except for less commenting.

### <p style="color:black;font-family:times new roman;font-size:120%;"> Import Required Libriaries <p/>

In [1]:
#import required libraries
!pip install utils
!pip install normalization
import normalization
import sklearn
from sklearn.feature_extraction.text import CountVectorizer
import utils
import numpy as np
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from pylab import *
from sklearn.manifold import MDS
from sklearn.metrics.pairwise import cosine_similarity
from matplotlib.font_manager import FontProperties
import pandas as pd
import string
import nltk
from collections import Counter
import pandas as pd
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.porter import PorterStemmer
porter = PorterStemmer()
from scipy import sparse

You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


You are using pip version 9.0.1, however version 9.0.2 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


### <p style="color:black;font-family:times new roman;font-size:120%;"> Get required data </p>
<p style="color:black;font-family:times new roman;font-size:140%;"> A dataset of about 100 different technical courses (containing core & electives) was created.</p>

In [2]:
#Read Data from xlsx file 
tech_course_descr_df = pd.read_excel('Management University_Syllabus_desp.xlsx', index_col=None, encoding='utf8')

#Function to stem list 
def stem_list(_list):
    
    """This function stems the contents of a given list and returns 
    a new list with stemmed words """
    
    stemmed_list = []
    for i in range(len(_list)):
        tokenize_list = _list[i].split()
        temp1 =[]
        for j in range(len(tokenize_list)):
            _stem = porter.stem(tokenize_list[j])
            temp1.append(_stem)
            if len(temp1) > 1:
                temp2 = " ".join(temp1)
            else:
                temp2 = "".join(temp1)
        stemmed_list.append(temp2)
    return stemmed_list

### <p style="color:black;font-family:times new roman;font-size:120%;"> Preprocessing Data </p>
<p style="color:black;font-family:times new roman;font-size:140%;"> This data is extracted into a pandas data frame. The data is then cleaned by case conversion and stemming. </p>

In [3]:
#Clean Data
tech_course_descr_df['Course Descriptions'] = tech_course_descr_df.apply(lambda _: '', axis=1)
tech_course_descr_df['Course Descriptions'] = tech_course_descr_df['Course Description'].str.lower()

In [4]:
tech_course_descr_df['Course Descriptions'][1]

'a highlight of the master of business analytics curriculum is the analytics capstone project, which gives students an opportunity to work on real-life data science problems with industry practitioners. over a period of seven months, students work in teams of two with sponsor organizations around the united states and abroad. in january, the student teams are matched with host organizations and assigned dedicated faculty advisors. from february to may, students work on the projects on campus at mit, meeting regularly with their faculty advisor and host organization. in the summer, students travel to the host organization for a full-time summer internship. finally in september, upon completion of the summer internship and submission of a final report and presentation, students earn their formal degree.\n\nsample companies:\n\nge appliances\ngroupm\nmckinsey & company\nnordstrom\nstubhub\nwalmart'

<p style="color:black;font-family:times new roman;font-size:140%;"> The following list is combination of two lists, first was from Kaggle data set and second, was from indeed job postings using TFID vectorizer. The combined list is used for further code. Both of these lists can be seen seperatey in the first "Job and skills cleaning" notebook<p/>

In [5]:
skills_list = ['monte carlo method',
 'recommendation engine',
 'association rule',
 'rule induction',
 'problem solving',
 'cluster analysi',
 'strategy',
 'factor analysi',
 'decision tree',
 'tabular data',
 'visualization',
 'database',
 'clustering',
 'ensemble method',
 'deep learning',
 'random forest',
 'evolutionary approache',
 'speech recognition',
 'markov logic networksnaive baye',
 'recommender system',
 'text analytic',
 'supervised machine learning',
 'investment banking',
 'computer vision',
 'bayesian method',
 'vendor management',
 'customer oriented',
 'mars',
#  'busine',
 'collaborative filtering',
 'neural network',
 'knn',
 'reinforcement learning',
 'architecture',
 'hmm',
 'proprietary algorithm',
 'gradient boosting',
 'data visualization',
 'outlier detection',
 'gan',
 'natural language processing',
 'time series analysi',
 'business intelligence',
 'pca',
 'svm',
 'business requirement',
 'managerial skillsdata driven',
 'neural net',
 'hidden markov models hmm',
 'a/b testing',
 'change management',
 'uplift modeling',
 'simulation',
 'adversarial learning',
 'pca dimensionality reduction',
 'enterprise',
 'stakeholder management',
 'lift analysi',
 'data analysi',
 'statistic',
 'markov logic network',
 'survival analysi',
 'commerce',
 'enterprise tool',
 'relationship management',
 'cross validation',
 'logistic regression',
 'bagging',
 'project coordination',
 'agile project management',
 'segmentation',
 'naive baye',
 'hidden markov model',
 'financial analysi',
 'link analysi',
 'bayesian technique',
 'evolutionary algorithm',
 'decis making',
 'data insight',
 'machine translation',
 'automation',
 'prescriptive modeling',
 'unsupervised learning',
 'gradient boosted machine',
 'mooc',
 'strategy management',
 'business proce',
 'business administration',
 'fraud detection',
 'regression',
 'cnn',
 'support vector machine',
 'rnn',
 'networking',
 'text mining',
 'boosting',
 'customer service',
 'asset management',
 'genetic & evolutionary algorithm',
 'social network analysi',
 'kaggle ranking',
 'project management',
 'time serie',
 'dimensionality reduction',
 'genetic algorithm']
skills_list = list(set(skills_list))
len(skills_list)

104

<p style="color:black;font-family:times new roman;font-size:140%;"> Now, the skills are compared with each course description to extract the matched skills. <p/>

In [6]:
#Function to extract matched skills
def _extract_matched_skills(text, skills):
    match_skills =[]
    for p in range(len(skills)):
        if skills[p] in text:
            match_skills.append(skills[p])
    match_skills = list(set(match_skills))
    return match_skills


#Extracting matched skills from each course description
tech_course_descr_df['Matched Skills'] = tech_course_descr_df.apply(lambda _: '', axis=1)
for i in range(len(tech_course_descr_df['Course Descriptions'])):
    matched_skills = _extract_matched_skills(tech_course_descr_df['Course Descriptions'][i], skills_list)
    tech_course_descr_df['Matched Skills'][i] = matched_skills


<p style="color:black;font-family:times new roman;font-size:140%;"> The frequnecy of the matched skills is calculated. Subsequently, a matrix based on the skills list is created. <p/>

In [7]:
#Create new coloumn for the number of matched skills 
tech_course_descr_df['Matched Skills Frequency'] = tech_course_descr_df.apply(lambda _: '', axis=1)

#Filling the matched skills frequency column with the frequency of matched job skills 
for i in range(len(tech_course_descr_df['Matched Skills'])):
    matched_skills_frequency = len(tech_course_descr_df['Matched Skills'][i])
    tech_course_descr_df['Matched Skills Frequency'][i] = matched_skills_frequency

#Function to create Matrix 
def create_matrix_row(skills, matched_skills):
    matrix_row =[]
    for i in range(len(skills)):
        if skills[i] in matched_skills:
            matrix_row += [1]
        else: 
            matrix_row += [0]
    return matrix_row

#Create new coloumn
tech_course_descr_df['Matrix Row'] = tech_course_descr_df.apply(lambda _: '', axis=1)

#Add Matrix row to curresponding row of dataframe
for i in range(len(tech_course_descr_df['Matched Skills'])):
    matrix_row = create_matrix_row(skills_list, tech_course_descr_df['Matched Skills'][i])
    tech_course_descr_df['Matrix Row'][i] = matrix_row

<p style="color:black;font-family:times new roman;font-size:140%;"> The relative frequencies of each skill based on the job descriptions which was created earlier is uploaded and extracted into a pandas data frame. Aditionally, a new matrix is created based on this relative frequency.<p/>

In [8]:
#Create new coloumn
tech_course_descr_df['Matched Skills Frequency'] = tech_course_descr_df.apply(lambda _: '', axis=1)


#Read Data from xlsx file 
rel_freq_df = pd.read_csv('Mgmt_relative_frequency.csv')

#Function to create relative frequnecy Matrix 
def create_rel_freq_matrix_row(skills, matched_skills, rel_freq):
    matrix_row =[]
    for i in range(len(skills)):
        if skills[i] in matched_skills:
            matrix_row += [rel_freq[i]]
        else: 
            matrix_row += [0]
    return matrix_row

#Create new coloumn
tech_course_descr_df['Relative Fequency Matrix Row'] = tech_course_descr_df.apply(lambda _: '', axis=1)


#Adding Matrix row to curresponding row of dataframe
for i in range(len(tech_course_descr_df['Matched Skills'])):
    matrix_row = create_rel_freq_matrix_row(rel_freq_df['list'], tech_course_descr_df['Matched Skills'][i], rel_freq_df['frequency'])
    tech_course_descr_df['Relative Fequency Matrix Row'][i] = matrix_row
    

<p style="color:black;font-family:times new roman;font-size:140%;"> A  <i> "Total Score" </i>  is given to each course based on the total relative frequency that each course has attained through matched skills. This completes the creation of the required dataframe. Analysis will be done using this dataframe. </p>

In [10]:
#Create new coloumn 
tech_course_descr_df['Total Course Score'] = tech_course_descr_df.apply(lambda _: '', axis=1)


#Adding total course score to corresponding course 
for i in range(len(tech_course_descr_df['Matched Skills'])):
    score = sum(tech_course_descr_df['Relative Fequency Matrix Row'][i])
    tech_course_descr_df['Total Course Score'][i] = score
    

tech_course_descr_df

,University,Program Title,Courses,Course Type,Course Description,Course Descriptions,Matched Skills,Matched Skills Frequency,Matrix Row,Relative Fequency Matrix Row,Total Course Score
0,Columbia University,Masters in Business Data Analytics,The Analytics Edge,Core,"In the last decade, the amount of data availab...","in the last decade, the amount of data availab...","[statistic, clustering, gan, logistic regressi...",,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.024282199, 0,...",0.207629
1,Columbia University,Masters in Business Data Analytics,ANALYTICS CAPSTONE PROJECT,Core,A highlight of the Master of Business Analytic...,a highlight of the master of business analytic...,[gan],,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.127974
2,Columbia University,Masters in Business Data Analytics,Data Mining: Finding the Data and Models that...,Elective,Provides an introduction to data mining and ma...,provides an introduction to data mining and ma...,"[fraud detection, neural network, database, as...",,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0.0, 0, 0, 0.000410172, 0, 0, 0, 0, 0, ...",0.101148
3,Columbia University,Masters in Business Data Analytics,Statistical Learning and Data Mining,Elective,Advanced introduction to the theory and applic...,advanced introduction to the theory and applic...,"[statistic, bayesian method, clustering, boost...",,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0024610...",0.0863822
4,Columbia University,Masters in Business Data Analytics,Statistical Thinking and Data Analysis,Elective,Introduces a rigorous treatment of statistical...,introduces a rigorous treatment of statistical...,"[regression, statistic, data analysi]",,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0826087
5,Columbia University,Masters in Business Data Analytics,Data Infrastructure,Core,Ensuring the validity and availability of data...,ensuring the validity and availability of data...,"[database, gan]",,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.050287121,...",0.178261
6,Columbia University,Masters in Business Data Analytics,Data Science for Business,Core,Deeply explore the interplay between data scie...,deeply explore the interplay between data scie...,"[clustering, supervised machine learning, text...",,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.0024610...",0.00287117
7,Columbia University,Masters in Business Data Analytics,Decision Analytics and Modeling,Core,Gain an overview of modeling frameworks and th...,gain an overview of modeling frameworks and th...,"[simulation, decision tree]",,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0.001804758, 0, 0, 0,...",0.00467596
8,Columbia University,Masters in Business Data Analytics,Data Visualization,Core,Explore techniques to effectively communicate ...,explore techniques to effectively communicate ...,"[data visualization, visualization]",,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.024282199, 0,...",0.0391304
9,Columbia University,Masters in Business Data Analytics,Business Fundamentals,Core,his course introduces you to a wide range of p...,his course introduces you to a wide range of p...,[strategy],,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0.068744873, 0, 0, 0, 0, 0,...",0.0687449


## we have initially created sparse 'feature matrix' using count vectorizer but we noticed a problem. for example we have a skill called 'data mining' the count vecotirizer is dividing this in to 'data' and 'mining' and due this the number of unique words obtained by count vectorizer is more than our inital list of algorithms. So we created our own sparse matrix using the colum 'Matrix Row'. Now the word 'data mining' is not sepearted during clustering and we cant get courses similar to each other on one cluster 

In [11]:
tech_course_descr_df['Matrix Row'] = tech_course_descr_df['Matrix Row'].apply(lambda x: np.array(x))

In [12]:
feature_matrix = np.array(tech_course_descr_df['Matrix Row'].tolist())

In [13]:
feature_matrix = sparse.csr_matrix(feature_matrix)

In [14]:
feature_matrix_dbscan = feature_matrix

### <p style="color:black;font-family:times new roman;font-size:120%;"> Application of Algorithm </p>
> #### <p style="color:black;font-family:times new roman;font-size:140%;"> Density Based Cluster Analysis <br> using DBSCAN (Density-Based Spatial Clustering of Applications with Noise) 

<p style="color:black;font-family:times new roman;font-size:140%;"> The text clustering
has an embedded process of unsupervised learning, where text group’s discovery
is performed without any prior information that could link each text to some
specific group </p> 

In [15]:
dbscan = DBSCAN(eps= 0.1, min_samples=2)

In [16]:
dbscan

DBSCAN(algorithm='auto', eps=0.1, leaf_size=30, metric='euclidean',
    metric_params=None, min_samples=2, n_jobs=1, p=None)

In [17]:
data = dbscan.fit(feature_matrix)
labels = data.labels_
#core_samples = np.zeros_like(labels, dtype = bool)
core_samples = data.core_sample_indices_
#core_samples[data.core_sample_indices_] = True

#Number of clusters formed 
n_clusters_ = len(set(labels)) - (1 if -1 in labels else 0)
print("Number of clusters formed:", n_clusters_)
core_samples

Number of clusters formed: 12


array([ 1,  5,  8, 10, 11, 12, 13, 14, 16, 21, 23, 26, 27, 28, 31, 32, 33,
       36, 37, 38, 39, 40, 42, 43, 45, 47, 48, 50, 52, 53, 54, 59, 61, 62,
       63, 64, 65, 66, 71, 73, 74, 75, 76, 77, 78, 79, 80, 81, 84, 85, 87,
       89, 90], dtype=int64)

In [18]:
#clusters = [data[labels == i] for i in range(n_clusters_)]
clusters = []
for i in range(n_clusters_):
    temp = dbscan.labels_[i]
    clusters.append(temp)
print(clusters)


outliers = data.labels_[-1]
outliers

[-1, 0, -1, -1, -1, 1, -1, -1, 2, -1, 3, 3]


10

In [19]:
y_pred_dbscan = dbscan.labels_

In [20]:
y_pred_df_dbscan = pd.DataFrame(y_pred_dbscan)

In [21]:
cluster_counts_dbscan = pd.DataFrame(y_pred_df_dbscan[0].value_counts())

In [22]:
cluster_counts_dbscan 

,0
-1,38
3,21
0,10
6,3
4,3
10,2
11,2
1,2
9,2
8,2


Main drawback 
- cannot deal with varying densities 
- sensitive to parameters
- suffers from curse of dimensionality 


### As you can see the above data frame, except for the category '-1' rest all were perfectly clustered. Now we use Kmeans on the '-1' category courses to cluster them furthur. By this we are taking advantages of both Algorithms

In [23]:
clustered_df_dbscan = tech_course_descr_df[['Courses','Matrix Row','Total Course Score','Matched Skills']]

In [24]:
clustered_df_dbscan["Cluster ID"] = y_pred_df_dbscan

C:\Users\Naga Raja Paidimarri\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [25]:
clustered_df_dbscan = clustered_df_dbscan.sort_values('Cluster ID')

In [26]:
clustered_df_kmeans = clustered_df_dbscan.sort_values('Cluster ID')

In [27]:
# clustered_df_kmeans.to_csv('clusteredmgmt.csv')

In [28]:
clustered_df_kmeans = clustered_df_kmeans.iloc[0:38,:]

In [29]:
clustered_df_kmeans 

,Courses,Matrix Row,Total Course Score,Matched Skills,Cluster ID
0,The Analytics Edge,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.207629,"[statistic, clustering, gan, logistic regressi...",-1
57,Introduction to Consulting and Advisory Thinking,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",0.100328,"[commerce, strategy]",-1
86,Business Intelligence with Visual Analytics (H...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",0.354307,"[data visualization, statistic, gan, data anal...",-1
6,Data Science for Business,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.00287117,"[clustering, supervised machine learning, text...",-1
3,Statistical Learning and Data Mining,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",0.0863822,"[statistic, bayesian method, clustering, boost...",-1
7,Decision Analytics and Modeling,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.00467596,"[simulation, decision tree]",-1
9,Business Fundamentals,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",0.0687449,[strategy],-1
88,Fundamentals of Systems Engineering for Engine...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0276456,[architecture],-1
22,Data Driven Decision Making,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.144545,"[business intelligence, regression, statistic,...",-1
82,Fundamentals of Analytics and Discovery Inform...,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0465956,"[statistic, decision tree]",-1


In [30]:
clustered_df_kmeans['Matrix Row'] =clustered_df_kmeans['Matrix Row'].apply(lambda x: np.array(x))
feature_matrix = np.array(clustered_df_kmeans['Matrix Row'].tolist())
feature_matrix = sparse.csr_matrix(feature_matrix)

In [31]:
feature_matrix

<38x104 sparse matrix of type '<class 'numpy.int32'>'
	with 125 stored elements in Compressed Sparse Row format>

In [32]:
# df_new2

In [33]:
for i in range(len(clustered_df_dbscan['Cluster ID'])):
    if clustered_df_dbscan['Matched Skills'][i] == []:
        print(clustered_df_dbscan['Courses'][i], [i])

Business Communication 1 [10]
Business Communication 2 [11]
Financial Risk Management [13]
Fixed Income Securities [14]
Applied Machine Learning for Financial Modeling and Forecasting [16]
Decision Under Risk [26]
Data Mining w/SAS Enterprise Miner [32]
MGMT 57200: Six Sigma & Quality Management [39]
MGMT 59000: Computing for Analytics [42]
MGMT 6870: Design: Social Networks & Engagements [43]
MGMT 59000: Spreadsheets & Macro Programming [45]
MGMT 59000: Web Data Analytics [47]
python programming [50]
Introduction to Business Analytics [52]
Customer Analytics [53]
 Problem Structuring [64]
 Big Data [66]
Application Domains [73]
Practicum Initiation [74]
Practicum Analysis [76]
Advanced Analytics and Practicum [81]


> #### <p style="color:black;font-family:times new roman;font-size:140%;"> K-Means Cluster Analysis <br> using centroid based clustering  

In [34]:
from sklearn.cluster import KMeans

# define the k-means clustering function
def k_means(feature_matrix, num_clusters):
    km = KMeans(n_clusters=num_clusters,max_iter=10000)
    km.fit(feature_matrix)
    clusters = km.labels_
    return km, clusters

# setting k value
num_clusters = 15

# get clusters and assigned the cluster labels to the movies
km_obj, clusters = k_means(feature_matrix=feature_matrix, num_clusters=num_clusters)
clustered_df_kmeans['Cluster'] = clusters

km_obj

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=10000,
    n_clusters=15, n_init=10, n_jobs=1, precompute_distances='auto',
    random_state=None, tol=0.0001, verbose=0)

In [35]:
from collections import Counter
# get the total number of courses per cluster
c = Counter(clusters)
print(c.items())

dict_items([(3, 1), (7, 5), (13, 1), (5, 4), (14, 1), (0, 3), (4, 7), (6, 1), (12, 1), (1, 3), (8, 1), (10, 2), (9, 4), (2, 2), (11, 2)])


In [36]:
clustered_df_kmeans = clustered_df_kmeans.sort_values('Cluster')

In [37]:
clustered_df_kmeans

,Courses,Matrix Row,Total Course Score,Matched Skills,Cluster ID,Cluster
49,MGMT 68400: Information Risk Management,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.155619,"[architecture, gan]",-1,0
67,Data Design and Representation,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.205906,"[database, architecture, gan]",-1,0
88,Fundamentals of Systems Engineering for Engine...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0276456,[architecture],-1,0
30,Revenue Management and Pricing,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0701395,"[commerce, segmentation, data analysi]",-1,1
19,Data Science for Business Analytics,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0336341,"[unsupervised learning, clustering, data analysi]",-1,1
20,Dealing with Data,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0816243,"[database, text analytic, data analysi]",-1,1
44,MGMT 59000: Predictive Analytics,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0.0540607,"[neural network, logistic regression, regressi...",-1,2
72,Machine Learning,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.045201,"[business intelligence, neural network, neural...",-1,2
0,The Analytics Edge,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.207629,"[statistic, clustering, gan, logistic regressi...",-1,3
69,Statistical Exploration and Reasoning,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0757998,"[statistic, data analysi]",-1,4


## Nowin the following code,The aim is to combine this clustered courses by kmeans with the dbscan data frame such that these kmeans data frame replaces the '-1' clustered category on the dbscan data frame. To make sure that the category numbers dont match, I multiplied the above kmeans cluster with -1 and conveted '0' to -15.

In [38]:
clustered_df_kmeans['Cluster ID'] = clustered_df_kmeans['Cluster ID']*clustered_df_kmeans['Cluster']

In [39]:
clustered_df_kmeans

,Courses,Matrix Row,Total Course Score,Matched Skills,Cluster ID,Cluster
49,MGMT 68400: Information Risk Management,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.155619,"[architecture, gan]",0,0
67,Data Design and Representation,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.205906,"[database, architecture, gan]",0,0
88,Fundamentals of Systems Engineering for Engine...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0276456,[architecture],0,0
30,Revenue Management and Pricing,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0701395,"[commerce, segmentation, data analysi]",-1,1
19,Data Science for Business Analytics,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0336341,"[unsupervised learning, clustering, data analysi]",-1,1
20,Dealing with Data,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0816243,"[database, text analytic, data analysi]",-1,1
44,MGMT 59000: Predictive Analytics,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0.0540607,"[neural network, logistic regression, regressi...",-2,2
72,Machine Learning,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.045201,"[business intelligence, neural network, neural...",-2,2
0,The Analytics Edge,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.207629,"[statistic, clustering, gan, logistic regressi...",-3,3
69,Statistical Exploration and Reasoning,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0757998,"[statistic, data analysi]",-4,4


In [40]:
for i in range(len(clustered_df_kmeans)):
    if clustered_df_kmeans.iloc[i,4] == 0:
        clustered_df_kmeans.iloc[i,4] = -15

In [41]:
clustered_df_kmeans 

,Courses,Matrix Row,Total Course Score,Matched Skills,Cluster ID,Cluster
49,MGMT 68400: Information Risk Management,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.155619,"[architecture, gan]",-15,0
67,Data Design and Representation,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.205906,"[database, architecture, gan]",-15,0
88,Fundamentals of Systems Engineering for Engine...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0276456,[architecture],-15,0
30,Revenue Management and Pricing,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0701395,"[commerce, segmentation, data analysi]",-1,1
19,Data Science for Business Analytics,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0336341,"[unsupervised learning, clustering, data analysi]",-1,1
20,Dealing with Data,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0816243,"[database, text analytic, data analysi]",-1,1
44,MGMT 59000: Predictive Analytics,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0.0540607,"[neural network, logistic regression, regressi...",-2,2
72,Machine Learning,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.045201,"[business intelligence, neural network, neural...",-2,2
0,The Analytics Edge,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.207629,"[statistic, clustering, gan, logistic regressi...",-3,3
69,Statistical Exploration and Reasoning,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0757998,"[statistic, data analysi]",-4,4


In [42]:
clustered_df_kmeans = clustered_df_kmeans.sort_values('Cluster ID')

In [43]:
clustered_df_kmeans = clustered_df_kmeans.drop('Cluster', axis=1)

In [44]:
clustered_df = clustered_df_dbscan

In [45]:
clustered_df

,Courses,Matrix Row,Total Course Score,Matched Skills,Cluster ID
0,The Analytics Edge,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.207629,"[statistic, clustering, gan, logistic regressi...",-1
25,Network Analytics,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0315833,"[a/b testing, commerce]",-1
56,Marketing and Quantitative Analysis,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.167104,"[commerce, segmentation, gan]",-1
29,Strategic Capstone,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",0.113536,"[strategy, statistic]",-1
30,Revenue Management and Pricing,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0701395,"[commerce, segmentation, data analysi]",-1
34,Business Analytics,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0854799,"[regression, statistic, data analysi, simulation]",-1
35,Advanced Business Analytics w/SAS,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0556194,"[logistic regression, time serie, regression, ...",-1
72,Machine Learning,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.045201,"[business intelligence, neural network, neural...",-1
70,Advanced Statistics,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0478261,"[time serie, statistic]",-1
69,Statistical Exploration and Reasoning,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0757998,"[statistic, data analysi]",-1


In [46]:
# clustered_df_kmeans

In [47]:
clustered_df.iloc[0:38,:] = clustered_df_kmeans.iloc[0:38,:]

In [48]:
clustered_df = clustered_df.sort_values('Cluster ID')

In [49]:
clustered_df 

,Courses,Matrix Row,Total Course Score,Matched Skills,Cluster ID
49,MGMT 68400: Information Risk Management,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.155619,"[architecture, gan]",-15
88,Fundamentals of Systems Engineering for Engine...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0276456,[architecture],-15
67,Data Design and Representation,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.205906,"[database, architecture, gan]",-15
3,Statistical Learning and Data Mining,"[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...",0.0863822,"[statistic, bayesian method, clustering, boost...",-14
86,Business Intelligence with Visual Analytics (H...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...",0.354307,"[data visualization, statistic, gan, data anal...",-13
83,Enterprise Software Architecture,"[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, ...",0.25603,"[project management, enterprise, gan, business...",-12
51,MGMT 59000: Production Scale Big Data Implemen...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.197047,"[visualization, statistic, gan]",-11
41,MGMT 59000: Big Data Technologies,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.15242,"[visualization, social network analysi, gan]",-11
46,MGMT 59000: Using R for Analytics,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0701395,"[visualization, data visualization, data analysi]",-10
24,Data Visualization,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",0.0421657,"[time serie, data visualization, visualization]",-10


In [50]:
clustered_df = clustered_df.drop('Matrix Row', axis=1)

In [51]:
clustered_df.to_csv('clusteredmgmtcourses.csv')

In [52]:
# clustered_df_kmeans.sort_values('Cluster').to_csv('clusteredmgmt2.csv')

### you can see the complete clustered courses in the 'clusteredmgmtcourses' file. There are around 20 clusters. Now we used a mix of Total score (based on relative frequency) and clustering to select courses for our circullum. our main is to pick courses from different clusters so that there wont be any repetition of algorithms in the circullum. But we also want to pick courses with high 'Total Course score' so that skills demanded in job market are covered.

In [53]:
# from sklearn.manifold import MDS
# from sklearn.metrics.pairwise import cosine_similarity

# #Get cluster details

# def get_cluster_data_dbscan(cluster_data, num_clusters, topn_features=10):
#     cluster_details = {}
#     # get courses belonging to each cluster
#     for cluster_num in range(num_clusters):
#         cluster_details[cluster_num] = {}
#         cluster_details[cluster_num]['cluster_num'] = cluster_num
#         courses = cluster_data[cluster_data['Cluster ID'] == cluster_num]['Courses'].values.tolist()
#         cluster_details[cluster_num]['Courses'] = courses
#     return cluster_details


# #Print data
# def print_cluster_data(cluster_data):
#     # print cluster details
#     for cluster_num, cluster_details in cluster_data.items():
#         print('Cluster {} details:'.format(cluster_num))
#         print('-'*20)
#         print()
#         print('Courses in this cluster:')
#         print(', '.join(cluster_details['Courses']))
#         print()
#         print('='*100)
#         print()
# #get dbscan cluster details
# dbscan_details = get_cluster_data_dbscan(clustered_df_dbscan, n_clusters_ , topn_features=10)
# #print dbscan cluster details
# print_cluster_data(dbscan_details)

# def plot_cluster_plot_points_dbscan(num_clusters, feature_matrix, cluster_data, course_data):
#     # build cosine distance matrix
#     cosine_distance = 1 - cosine_similarity(feature_matrix)
#     # dimensionality reduction using MDS
#     mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
#     # get coordinates of clusters in new low-dimensional space
#     plot_positions = mds.fit_transform(cosine_distance)
#     x_pos, y_pos = plot_positions[:, 0], plot_positions[:, 1]
#     # map each unique cluster label with its coordinates and courses
#     cluster_plot_df = pd.DataFrame({'x': x_pos,'y': y_pos,'label': course_data['Cluster ID'].values.tolist(),'Course': course_data['Courses'].values.tolist()})
#     cluster_plot_df.to_csv('Technical_DBSCAN.csv')
    
    
# #Get cluster details

# def get_cluster_data_kmeans(cluster_data, num_clusters, topn_features=10):
#     cluster_details = {}
#     # get courses belonging to each cluster
#     for cluster_num in range(num_clusters):
#         cluster_details[cluster_num] = {}
#         cluster_details[cluster_num]['cluster_num'] = cluster_num
#         courses = cluster_data[cluster_data['Cluster'] == cluster_num]['Courses'].values.tolist()
#         cluster_details[cluster_num]['Courses'] = courses
#     return cluster_details

# def plot_cluster_plot_points_kmeans(num_clusters, feature_matrix, cluster_data, course_data):
#     # build cosine distance matrix
#     cosine_distance = 1 - cosine_similarity(feature_matrix)
#     # dimensionality reduction using MDS
#     mds = MDS(n_components=2, dissimilarity="precomputed", random_state=1)
#     # get coordinates of clusters in new low-dimensional space
#     plot_positions = mds.fit_transform(cosine_distance)
#     x_pos, y_pos = plot_positions[:, 0], plot_positions[:, 1]
#     # map each unique cluster label with its coordinates and courses
#     cluster_plot_df = pd.DataFrame({'x': x_pos,'y': y_pos,'label': course_data['Cluster'].values.tolist(),'Course': course_data['Courses'].values.tolist()})
#     cluster_plot_df.to_csv('Technical_KMEANS.csv')

# #get plotting points for dbscan
# plot_cluster_plot_points_dbscan(n_clusters_, feature_matrix_dbscan, dbscan_details, clustered_df_dbscan)

# #get kmeans cluster details
# k_means_details = get_cluster_data_kmeans(clustered_df_kmeans, num_clusters, topn_features=10)

# #print kmeans cluster details
# print_cluster_data(dbscan_details)

# #get plotting points for dbscan
# plot_cluster_plot_points_kmeans(num_clusters, feature_matrix, k_means_details, clustered_df_kmeans)